In [79]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
from intvalpy import lineqs
import sympy
import numpy as np
import os
import json
from scipy.optimize import linprog
from ipywidgets import interact, widgets
from functools import lru_cache
#plt.rcParams['text.usetex'] = True

# Mixed Integer Linear Program (MILP)
$$
\begin{align}
&\min_{x} && c^T x\\
&\text{s.t.} && Ax \leq b\\
& && Bx = d\\
& && x \geq 0\\
& && x \in \mathbb{Z}^{n_1} \times \mathbb{R}^{n-n_1}
\end{align}
$$

In [69]:
def run_cutting_planes(filepath):
    b = os.popen(filepath).read()
    print(b)

def load_data(milp_filename, cp_filename):
    return json.load(open(milp_filename)), json.load(open(cp_filename))

exe_file = '/Users/tobiaskohler/Uni/CuttingPlanes/build-CuttingPlanes-Desktop_arm_darwin_generic_mach_o_64bit-Debug/CuttingPlanes'
run_cutting_planes('/Users/tobiaskohler/Uni/CuttingPlanes/build-CuttingPlanes-Desktop_arm_darwin_generic_mach_o_64bit-Debug/CuttingPlanes HelloWorld')
data_milp, data_cp = load_data('milp.json', 'cp.json')
print(data_milp, data_cp)

# Access the system data
n, n1, m, p = data_milp['n'], data_milp['n1'], data_milp['m'], data_milp['p']
A = -np.array(data_milp['A'], dtype=np.float64)
b = -np.array(data_milp['b'], dtype=np.float64)
c = np.array(data_milp['c'], dtype=np.float64)
B = np.array(data_milp['B'], dtype=np.float64)
d = np.array(data_milp['d'], dtype=np.float64)
sols = data_cp['sols']
A_cuts = -np.array(data_cp['cuts_coeffs'], dtype=np.float64)
b_cuts = -np.array(data_cp['cuts_rhs'], dtype=np.float64)
n_cuts = len(b_cuts)

assert(A.shape[0] == m and A.shape[1] == n)
assert(n_cuts==len(sols)-1)
assert(n == 2)

# Insert nonnegative constraints for visualisation
m += 2
A = np.vstack([[1,0], [0,1], A])
b = np.append([0,0], b)

HelloWorld

{'A': [[3.0, 2.0], [-3.0, 2.0]], 'B': [], 'b': [6.0, 0.0], 'c': [0.0, -1.0], 'd': [], 'm': 2, 'n': 2, 'n1': 2, 'p': 0} {'cuts_coeffs': [[0.0, 1.0], [-2.0, 2.0]], 'cuts_rhs': [1.0, 0.0], 'sols': [[1.0, 1.5], [0.6666666666666666, 1.0], [1.0, 1.0]]}


In [121]:
vs = lineqs(A, b, show=False)
xlim, ylim = [-0, 1.1*np.max(vs[:,0])], [-0, 1.1*np.max(vs[:,1])]

def coeff_str(coeff, sym):
    if coeff==0:
        return ''
    p = '' if coeff >= 0 else '-'
    c = abs(int(coeff)) if coeff%1==0 else abs(coeff)
    return p + (str(c)+'$\cdot$' if c != 1 else '') + sym

def ineq_str(A, b, i):
    cx, cy =-A[i,0], -A[i,1]
    s = [t for t in [coeff_str(cx, 'x'), coeff_str(cy, 'y')] if t]
    return ' + '.join(s) + ' $\leq$ ' + str(int(-b[i])) if b[i]%1==0 else str(-b[i])

def plot_lp(title, c, A, b, B, d, sol=None, cut=False):

    ratio = (xlim[1]-xlim[0])/(ylim[1]-ylim[0])
    plt.figure(figsize=(10,10/ratio))
    assert(len(c) == 2)
    m = len(b)

    # Generate Meshgrid of Points
    x_vals = np.linspace(xlim[0], xlim[1], 400)
    y_vals = np.linspace(ylim[0], ylim[1], 400)
    X, Y = np.meshgrid(x_vals, y_vals)

    # Plot Objective Function
    Z = c[0]*X + c[1]*Y
    plt.imshow(Z, extent=[xlim[0], xlim[1], ylim[0], ylim[1]], origin='lower', cmap='viridis', alpha=0.3)
    plt.contour(X, Y, Z, levels=20, cmap='viridis', alpha=0.1)

    # Plot Inequality Constraints
    for i in range(m):
        if A[i,1] != 0:
            plt.plot(x_vals, (b[i] - A[i,0]*x_vals) / A[i,1], 'r-' if (cut and i==m-1) else 'g-', lw=2, label=ineq_str(A,b,i))
        elif A[i,0] != 0:
            plt.axvline(x=b[i] / A[i, 0], color='r' if (cut and i==m-1) else 'g', linestyle='-', lw=2, label=ineq_str(A,b,i))

    # Shade Feasible Region
    feasible_cmap = matplotlib.colors.ListedColormap(['white', 'lime'])
    feasible_region = np.all(A[:,0,None,None]*X + A[:,1,None,None]*Y >= b[:,None,None], axis=0)
    plt.contourf(X, Y, feasible_region, cmap=feasible_cmap, alpha=0.2)

    # Plot Integer Points
    x_int = np.arange(np.ceil(xlim[0]), np.floor(xlim[1])+1, 1)
    y_int = np.arange(np.ceil(ylim[0]), np.floor(ylim[1])+1, 1)
    X_int, Y_int = np.meshgrid(x_int, y_int)
    plt.scatter(X_int, Y_int, s=40, color='blue', label='Integers')

    # Plot Optimal Solution
    if sol is not None:
        plt.plot(sol[0],sol[1],'ro', markersize=10, label='x=(%g, %g)' % (sol[0],sol[1]))

    # Labels and Stuff
    plt.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(title)
    plt.grid(color='gray', linestyle='-', linewidth=1)
    plt.xticks(np.arange(np.floor(xlim[0]), xlim[1], 1))
    plt.yticks(np.arange(np.floor(ylim[0]), ylim[1], 1))
    plt.xlim(xlim)
    plt.ylim(ylim)

    # Show plot
    plt.show()

In [122]:

def plot_cutting_planes(iter):
    i = iter//2
    j = iter%2

    if i==0:
        plot_lp(r'$\frac{1}{2}$', c, A, b, B, d, sol=None if j==0 else sols[0]) 
        return

    sol = sols[i-1] if j==0 else sols[i]

    title='Cutting Plane' if j==0 else 'Solution'
    plot_lp(title, c, np.vstack([A, A_cuts[:i]]), np.append(b, b_cuts[:i]), B, d, sol=sol, cut=j==0)

interact(plot_cutting_planes, iter=widgets.IntSlider(min=0, max=2*n_cuts+1, step=1, value=-2, description='Show Step'))

interactive(children=(IntSlider(value=0, description='Show Step', max=5), Output()), _dom_classes=('widget-int…

<function __main__.plot_cutting_planes(iter)>